In [1]:
import polars as pl

Начнем с обработки данных и соберем их в единую таблицу

С headline cpi все нормально, просто закастуем нужны типы данных и разделим год и квартал

In [2]:
headline_cpi = pl.read_csv('data/headline_cpi.csv')
headline_cpi = headline_cpi.filter(pl.col('Title').str.contains('Q1|Q2|Q3|Q4'))
headline_cpi = headline_cpi.select(pl.col('Title').alias('quarter_str'), pl.col('CPI ANNUAL RATE 00: ALL ITEMS 2015=100').cast(pl.Float64).alias('headline_cpi'))
headline_cpi = headline_cpi.with_columns([
    pl.col("quarter_str").str.extract(r"(\d{4})", 1).cast(pl.Int32).alias("year"),
    pl.col("quarter_str").str.extract(r"Q(\d)", 1).cast(pl.Int8).alias("quarter")
])
headline_cpi = headline_cpi.drop('quarter_str')
headline_cpi

headline_cpi,year,quarter
f64,i32,i8
5.0,1989,1
5.3,1989,2
5.1,1989,3
5.5,1989,4
5.8,1990,1
…,…,…
6.7,2023,3
4.2,2023,4
3.5,2024,1


C core_cpi все сложнее, так как данные в формате m-on-m, то есть изменение за месяц. Во-первых, надо разбить столбец даты на год и квартал (тут сложность в том, что в таблице указывается инфляция за предыдущий период, то есть если дата - 2000-10-01, то инфляция за сентябрь). Во-вторых, надо перемножить инфляцию в каждом квартале, чтобы получить общую за квартал, а потом аннуализировать ее.

In [3]:
core_cpi = pl.read_csv('data/core_cpi_m-on-m.csv')
core_cpi = core_cpi.with_columns(pl.col('DATE').str.strptime(pl.Date, format="%Y-%m-%d")).rename({'GBRCPHPLA01GPM' : 'core_cpi'})
core_cpi = core_cpi.with_columns([
    pl.col("DATE").dt.year().alias("year"),
    ((pl.col("DATE").dt.month() - 2) // 3 + 1).alias("quarter")
])
core_cpi = core_cpi.with_columns(pl.when(pl.col('quarter') == 0)
                      .then(pl.lit(4))
                      .otherwise(pl.col('quarter'))
                      .alias('quarter'))
core_cpi = core_cpi.with_columns(pl.when(pl.col('DATE').dt.month() == 1).
                      then(pl.col('year') - 1).
                      otherwise(pl.col('year')).
                      alias('year'))
core_cpi = core_cpi.drop('DATE').group_by(['year', 'quarter']).agg(((pl.col('core_cpi') / 100 + 1).product() - 1) * 100).sort(['year', 'quarter'])
core_cpi = core_cpi.with_columns((((pl.col('core_cpi') / 100 + 1).pow(4) - 1) * 100).round(1))
core_cpi

year,quarter,core_cpi
i32,i8,f64
1988,1,8.6
1988,2,4.5
1988,3,6.0
1988,4,2.2
1989,1,8.9
…,…,…
2022,4,0.0
2023,1,14.3
2023,2,5.2


Склеим core и headline cpi

In [4]:
overall_data = headline_cpi.join(core_cpi, on=['year', 'quarter'], how='left')

Информацию об инфляционных ожиданиях мы взяли из опросов на сайте Bank of England (ЦБ местного розлива). В таблице есть разные данные, но мы взяли медианное значение ответов на вопрос "How much would you expect prices in the shops generally to change over the next 12 months?".

In [5]:
# !pip install fastexcel

In [6]:
inf_expectations = pl.read_excel('data/inflation_expectations.xlsx')
df = pl.concat([inf_expectations.filter(pl.col('PUBLIC ATTITUDES TO INFLATION') == 'SUMMARY RESULTS '), inf_expectations.filter(pl.col('PUBLIC ATTITUDES TO INFLATION') == 'Median')[1]]).transpose()
df.columns = df.row(0)
inf_expectations = df[1::]
inf_expectations = inf_expectations.with_columns(pl.col('SUMMARY RESULTS ').str.strptime(pl.Date, format="%Y-%m-%d %H:%M:%S"), pl.col('Median').cast(pl.Float64).round(1)).with_columns(pl.col('SUMMARY RESULTS ').dt.year().alias('year'), (pl.col('SUMMARY RESULTS ').dt.month() // 3 + 1).alias('quarter'))
inf_expectations = inf_expectations.drop('SUMMARY RESULTS ').rename({'Median' : 'inf_expectation'})
inf_expectations

inf_expectation,year,quarter
f64,i32,i8
1.5,1999,4
2.3,2000,1
2.4,2000,2
2.2,2000,3
2.3,2000,4
…,…,…
3.6,2023,3
3.3,2023,4
3.0,2024,1


In [7]:
overall_data = overall_data.join(inf_expectations, on=['year', 'quarter'], how='left')

GDP так же как и другие данные аннуализируем сложным процентом

In [8]:
gdp = pl.read_csv('data/gdp_q-on-q.csv')
gdp = gdp.filter(pl.col("Title").str.contains('Q1|Q2|Q3|Q4'))
gdp = gdp.with_columns([
    pl.col("Title").str.extract(r"(\d{4})", 1).cast(pl.Int32).alias("year"),
    pl.col("Title").str.extract(r"Q(\d)", 1).cast(pl.Int8).alias("quarter")
])
gdp = gdp.rename({'Gross Domestic Product: Quarter on Quarter growth: CVM SA %' : 'gdp'}).drop("Title").with_columns(pl.col("gdp").cast(pl.Float64))
gdp = gdp.with_columns((((pl.col('gdp') / 100 + 1).pow(4) - 1) * 100).round(1))
gdp

gdp,year,quarter
f64,i32,i8
0.0,1955,2
8.2,1955,3
-2.4,1955,4
4.5,1956,1
-0.4,1956,2
…,…,…
-0.4,2023,3
-1.2,2023,4
2.8,2024,1


In [9]:
overall_data = overall_data.join(gdp, on=['year', 'quarter'], how='right')

Output gap с годичной переодичностью, так как нет данных с большей частотностью. Надо держать в уме, что от квартала его значение в таблице не меняется.

In [10]:
output_gap = pl.read_excel('data/output_gap_in_it.xlsx')
output_gap = output_gap.filter([pl.col('Economy Name') == 'United Kingdom', pl.col('Indicator') == 'Output gap in percent of potential GDP, Percent of potential GDP', pl.col('Attribute 1') == 'Actual']).transpose(include_header=True)[14:-7]
output_gap.columns = ['year', 'output_gap']
output_gap = output_gap.with_columns(pl.col('year').cast(pl.Int32), pl.col('output_gap').cast(pl.Float64))
output_gap

year,output_gap
i32,f64
1986,1.08
1987,2.76
1988,3.43
1989,4.02
1990,2.3
…,…
2018,-0.31
2019,-0.04
2020,-3.59


In [11]:
overall_data = overall_data.join(output_gap, on='year', how='left')

Ну и наконец безработица по привычной схеме

In [12]:
unemployment = pl.read_csv('data/unemployment.csv')
unemployment = unemployment.filter(pl.col("Title").str.contains('Q1|Q2|Q3|Q4'))
unemployment = unemployment.with_columns([
    pl.col("Title").str.extract(r"(\d{4})", 1).cast(pl.Int32).alias("year"),
    pl.col("Title").str.extract(r"Q(\d)", 1).cast(pl.Int8).alias("quarter")
])
unemployment = unemployment.rename({'Unemployment rate (aged 16 and over, seasonally adjusted): %' : 'unemployment'}).drop("Title").with_columns(pl.col('unemployment').cast(pl.Float64))
unemployment

unemployment,year,quarter
f64,i32,i8
3.8,1971,1
4.1,1971,2
4.2,1971,3
4.4,1971,4
4.5,1972,1
…,…,…
4.1,2023,3
3.8,2023,4
4.3,2024,1


In [13]:
overall_data = overall_data.join(unemployment, on=['year', 'quarter'], how='left')
overall_data = overall_data.drop_nulls()
overall_data

headline_cpi,core_cpi,inf_expectation,gdp,year,quarter,output_gap,unemployment
f64,f64,f64,f64,i32,i8,f64,f64
1.1,-4.1,1.5,5.7,1999,4,-0.88,5.8
0.8,3.7,2.3,4.9,2000,1,0.46,5.8
0.6,-3.1,2.4,2.8,2000,2,0.46,5.5
0.8,4.8,2.2,2.4,2000,3,0.46,5.3
1.0,-3.1,2.3,2.4,2000,4,0.46,5.2
…,…,…,…,…,…,…,…
4.9,2.5,3.2,6.1,2021,4,0.53,4.2
6.2,9.8,4.3,2.8,2022,1,1.81,3.9
9.2,5.2,4.6,1.2,2022,2,1.81,3.9


Так у нас выглядит итоговая табличка. Поскольку данные об ожидаемой инфляции наичнаются с 2000-го года, то отсальные ряды придется подрезать. Но на всякий случай сохраним исходные таблицы.

In [14]:
overall_data.write_csv('clear_data/overall_data.csv')
headline_cpi.write_csv('clear_data/headline_cpi_clear.csv')
core_cpi.write_csv('clear_data/core_cpi_clear.csv')
inf_expectations.write_csv('clear_data/inflation_expectations_clear.csv')
gdp.write_csv('clear_data/gdp_clear.csv')
output_gap.write_csv('clear_data/output_gap_clear.csv')
unemployment.write_csv('clear_data/unemployment_clear.csv')